In [1]:
import pandas as pd
import sqlite3

In [8]:
customer= pd.read_csv('customer.csv')

cust_services= pd.read_csv('cust_services.csv')

cust_loc= pd.read_csv('cust_loc.csv')

cust_churn= pd.read_csv('cust_churn.csv')

cust_account= pd.read_csv('cust_account.csv')

In [9]:
customer.head(3)

,CustomerID,Gender,SeniorCitizen,Partner,Dependents
0,3668-QPYBK,Male,No,No,No
1,9237-HQITU,Female,No,No,Yes
2,9305-CDSKC,Female,No,No,Yes


In [10]:
cust_services.head(3)

,Cust_ID,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies
0,3668-QPYBK,Yes,No,DSL,Yes,Yes,No,No,No,No
1,9237-HQITU,Yes,No,Fiber optic,No,No,No,No,No,No
2,9305-CDSKC,Yes,Yes,Fiber optic,No,No,Yes,No,Yes,Yes


In [11]:
cust_loc.head(3)

,Cust_ID,State,Latitude,Longitude,ZipCode
0,3668-QPYBK,California,33.964131,-118.272783,90003
1,9237-HQITU,California,34.059281,-118.307420,90005
2,9305-CDSKC,California,34.048013,-118.293953,90006


In [12]:
cust_churn.head(3)

,Id,Churn
0,3668-QPYBK,Yes
1,9237-HQITU,Yes
2,9305-CDSKC,Yes


In [13]:
cust_account.head(3)

,Account_id,Tenure,Contract,PaymentMethod,PaperlessBilling,MonthlyCharges,TotalCharges
0,3668-QPYBK,2,Month-to-month,Mailed check,Yes,53.85,108.15
1,9237-HQITU,2,Month-to-month,Electronic check,Yes,70.70,151.65
2,9305-CDSKC,8,Month-to-month,Electronic check,Yes,99.65,820.5


In [14]:
cust_account.head()     # So all the dfs has same no. of rows, can be merged..
cust_services.shape
customer.shape

(7043, 5)

In [15]:
#### Creating Database

In [16]:
conn= sqlite3.connect('churn.db')
cur= conn.cursor()

In [17]:
# we gonna make table by converting dataframe to table
# sqlite considers datatpe of dataframe and chooses suitable datatype in sql

In [78]:
# create customer table
customer.to_sql('customer', conn, if_exists= 'replace', index= False)

In [ ]:
# create customer services
cust_services.to_sql('cust_services', conn, if_exists= 'replace', index= False)

In [ ]:
# create customer location table
cust_loc.to_sql('cust_loc', conn, if_exists= 'replace', index= False)

In [ ]:
# create customer account table
cust_account.to_sql('cust_account', conn, if_exists= 'replace', index= False)

In [ ]:
# create customer churn table
cust_churn.to_sql('cust_churn', conn, if_exists= 'replace', index= False)

In [ ]:
# INFO ABOUT TABLE 

In [ ]:
l= [customer, cust_services, cust_loc, cust_account, cust_churn]
a=[]
for i in l:
    a.append(cur.execute('PRAGMA table_info(i)').fetchall())

In [ ]:
cur.execute('PRAGMA table_info(cust_churn)').fetchall()

### Creating ER diagram for churn database

In [ ]:
def table_infofun(conn, cur):
    #fetch all colms of the database table
    tables= cur.execute('select name from sqlite_master where type= "table" ').fetchall()
    for table_name in tables:
        table_name= table_name[0]
        table= pd.read_sql_query(f'select * from {table_name} limit 0', conn)
        print(table_name)
        for col in table.columns:
            print('\t' + col)
            print()

In [ ]:
table_infofun(conn,cur)

#### Now we'll do analysis on our database tables using SQL and gain some insight

    Exploratory data analysis in SQLite

### Joining all Tables

In [ ]:
conn= sqlite3.connect('churn.db')
cur= conn.cursor()

cur.execute('alter table cust_loc rename column Cust_ID to CustomerID')

In [33]:
cur.execute('PRAGMA table_info(cust_loc)')
cur.fetchall()        # successfully renamed Colm 

[(0, 'CustomerID', 'TEXT', 0, None, 0),
 (1, 'State', 'TEXT', 0, None, 0),
 (2, 'Latitude', 'REAL', 0, None, 0),
 (3, 'Longitude', 'REAL', 0, None, 0),
 (4, 'ZipCode', 'INTEGER', 0, None, 0)]

In [ ]:
cur.execute('select * from customer inner join cust_loc using (CustomerID)').fetchone()
# first row of joined table

In [ ]:
## we can do for all tables to join all tables in our SQL database

In [73]:
#### To Join dataframes first rename columns to have single ID colm
### Renaming colms of difft tables
customer.rename(columns= {'CustomerID': 'customerID'}, inplace= True)
cust_loc.rename(columns= {'Cust_ID':'customerID'},inplace= True)
cust_services.rename(columns= {'Cust_ID':'customerID'},inplace= True)
cust_account.rename(columns= {'Account_id':'customerID'},inplace= True)
cust_churn.rename(columns= {'Id':'customerID'},inplace= True)

In [51]:
cust_loc.head()
cust_services.head()
cust_account.head()
cust_churn.shape

(7043, 2)

In [72]:
customer.head()

,CustomerID,Gender,SeniorCitizen,Partner,Dependents
0,3668-QPYBK,Male,No,No,No
1,9237-HQITU,Female,No,No,Yes
2,9305-CDSKC,Female,No,No,Yes
3,7892-POOKP,Female,No,Yes,Yes
4,0280-XJGEX,Male,No,No,Yes


In [64]:
# Now we're are gonna concat join: inner, dataframe on common colm(customerID)
l_cust= [customer, cust_loc, cust_services, cust_account, cust_churn]

In [9]:
# we can merge only 2 dataframes at a time, so let's use HIGHER ORDER FUN. Reduce()
from functools import reduce

In [33]:
# COMBINED DATAFRAME
churn_all= reduce(lambda x,y : pd.merge(x,y, on=['customerID']), l_cust)

In [32]:
cust_services.shape

(7043, 10)

In [77]:
churn_all.head()

,customerID,Gender,SeniorCitizen,Partner,Dependents,State,Latitude,Longitude,ZipCode,PhoneService,...,TechSupport,StreamingTV,StreamingMovies,Tenure,Contract,PaymentMethod,PaperlessBilling,MonthlyCharges,TotalCharges,Churn
0,3668-QPYBK,Male,No,No,No,California,33.964131,-118.272783,90003,Yes,...,No,No,No,2,Month-to-month,Mailed check,Yes,53.85,108.15,Yes
1,9237-HQITU,Female,No,No,Yes,California,34.059281,-118.307420,90005,Yes,...,No,No,No,2,Month-to-month,Electronic check,Yes,70.70,151.65,Yes
2,9305-CDSKC,Female,No,No,Yes,California,34.048013,-118.293953,90006,Yes,...,No,Yes,Yes,8,Month-to-month,Electronic check,Yes,99.65,820.5,Yes
3,7892-POOKP,Female,No,Yes,Yes,California,34.062125,-118.315709,90010,Yes,...,Yes,Yes,Yes,28,Month-to-month,Electronic check,Yes,104.80,3046.05,Yes
4,0280-XJGEX,Male,No,No,Yes,California,34.039224,-118.266293,90015,Yes,...,No,Yes,Yes,49,Month-to-month,Bank transfer (automatic),Yes,103.70,5036.3,Yes


In [74]:
# USING CONCAT TABLES in LIST
churn_all= pd.concat(l_cust, join= 'inner', axis= 1)

In [75]:
churn_all= churn_all.loc[:, ~churn_all.columns.duplicated()]

In [76]:
churn_all.shape

(7043, 25)

WE HAVE A DATAFRAME WITH all tables but only unique colms

In [79]:
churn_all.to_sql('churn_all', conn, if_exists= 'replace', index= False)